# Setting up our Schema

Spark can automatically create a schema for CSV files, but ours don't have headings. Let's set this up here:

In [1]:
from pyspark.sql.types import StructType, StructField, FloatType, LongType, StringType

feats = []
f = open('features.txt')
for line_num, line in enumerate(f):
    if line_num == 0:
        # Timestamp
        feats.append(StructField(line.strip(), LongType(), True))
    elif line_num == 1:
        # Geohash
        feats.append(StructField(line.strip(), StringType(), True))
    else:
        # Other features
        feats.append(StructField(line.strip(), FloatType(), True))
    
schema = StructType(feats)

print(schema)


StructType(List(StructField(Timestamp,LongType,true),StructField(Geohash,StringType,true),StructField(geopotential_height_lltw,FloatType,true),StructField(water_equiv_of_accum_snow_depth_surface,FloatType,true),StructField(drag_coefficient_surface,FloatType,true),StructField(sensible_heat_net_flux_surface,FloatType,true),StructField(categorical_ice_pellets_yes1_no0_surface,FloatType,true),StructField(visibility_surface,FloatType,true),StructField(number_of_soil_layers_in_root_zone_surface,FloatType,true),StructField(categorical_freezing_rain_yes1_no0_surface,FloatType,true),StructField(pressure_reduced_to_msl_msl,FloatType,true),StructField(upward_short_wave_rad_flux_surface,FloatType,true),StructField(relative_humidity_zerodegc_isotherm,FloatType,true),StructField(categorical_snow_yes1_no0_surface,FloatType,true),StructField(u-component_of_wind_tropopause,FloatType,true),StructField(surface_wind_gust_surface,FloatType,true),StructField(total_cloud_cover_entire_atmosphere,FloatType,tru

# Creating a Dataframe

Let's load our CSV into a 'dataframe' - Spark's abstraction for working with tabular data (built on top of RDDs)

In [2]:
df_tiny = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:15000/nam_tiny.tdv')
df_small = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:15000/nam_s/*')

In [3]:
df_full = spark.read.format('csv').option('sep', '\t').schema(schema).load('hdfs://orion11:15000/nam/*')

In [8]:
df_full.take(1)

KeyboardInterrupt: 

# Playtime

In [30]:
really_hot = df.filter(df.temperature_surface > 320).count()
print(really_hot)

hot_and_humid = df.filter(df.temperature_surface > 313).filter(df.relative_humidity_zerodegc_isotherm > .8).count()
print(hot_and_humid)

KeyboardInterrupt: 

In [5]:
df.filter(df.snow_cover_surface > .85).take(5)

[Row(Timestamp=1426377600000, Geohash='cf7ecr4h2ps0', geopotential_height_lltw=136.53125, water_equiv_of_accum_snow_depth_surface=77.0, drag_coefficient_surface=0.0, sensible_heat_net_flux_surface=-39.57763671875, categorical_ice_pellets_yes1_no0_surface=0.0, visibility_surface=24221.587890625, number_of_soil_layers_in_root_zone_surface=3.0, categorical_freezing_rain_yes1_no0_surface=0.0, pressure_reduced_to_msl_msl=99602.0, upward_short_wave_rad_flux_surface=6.625, relative_humidity_zerodegc_isotherm=34.0, categorical_snow_yes1_no0_surface=0.0, u-component_of_wind_tropopause=27.527877807617188, surface_wind_gust_surface=16.158788681030273, total_cloud_cover_entire_atmosphere=100.0, upward_long_wave_rad_flux_surface=314.0560302734375, land_cover_land1_sea0_surface=1.0, vegitation_type_as_in_sib_surface=18.0, v-component_of_wind_pblri=12.31915283203125, albedo_surface=38.75, lightning_surface=0.0, ice_cover_ice1_no_ice0_surface=0.0, convective_inhibition_surface=-0.65234375, pressure_su

# SQL

In [3]:
from pyspark.sql import functions as F

In [4]:
# Creating an SQL 'table'
df_tiny.createOrReplaceTempView("nam_tiny")
df_small.createOrReplaceTempView("nam_small")

In [5]:
df_full.createOrReplaceTempView("nam_full")

#### Unknown Feature
I didn't know what albedo was, so I looked at its summary statistics. Still unsure, I looked up the definition: 'the proportion of the incident light or radiation that is reflected by a surface, typically that of a planet or moon.'

In [11]:
df.describe('albedo_surface').show()

KeyboardInterrupt: 

#### Hot Hot Hot
The hottest tempurature in the dataset is at {location} at {time}. Looking at the other highest tempuratures, it {does/not} appear to be an anomaly. 

In [40]:
tempminmax = spark.sql("SELECT MIN(temperature_surface) as mintemp, MAX(temperature_surface) as maxtemp FROM nam_small").collect()

In [101]:
df.select(min(df.temperature_surface), max(df.temperature_surface)).first()[1]

306.4980163574219

In [48]:
hottest_record = df.orderBy("temperature_surface", ascending=False).first()
df.orderBy("temperature_surface", ascending=False).select("temperature_surface").head(10)

[Row(temperature_surface=306.4980163574219),
 Row(temperature_surface=301.3730163574219),
 Row(temperature_surface=299.9980163574219),
 Row(temperature_surface=299.9980163574219),
 Row(temperature_surface=299.7480163574219),
 Row(temperature_surface=299.6230163574219),
 Row(temperature_surface=299.4980163574219),
 Row(temperature_surface=299.1230163574219),
 Row(temperature_surface=298.9980163574219),
 Row(temperature_surface=298.8730163574219)]

In [99]:
df.describe('temperature_surface').show()

+-------+-------------------+
|summary|temperature_surface|
+-------+-------------------+
|  count|                100|
|   mean|  284.9017663574219|
| stddev| 13.002025568205239|
|    min|          247.49802|
|    max|          306.49802|
+-------+-------------------+



In [97]:
from datetime import datetime, timezone
timestamp = hottest_record['Timestamp'] / 1000
datetime.fromtimestamp(timestamp, timezone.utc)

datetime.datetime(2015, 3, 15, 0, 0, tzinfo=datetime.timezone.utc)

In [13]:
hottest_record['Geohash']

'9qd23ynghrrz'

In [194]:
rdd = df.select("temperature_surface").rdd

#rdd.map(lambda row : row[0]).collect()
#rdd.map(lambda row : row.temperature_surface).collect()
hist_rdd = df.rdd.map(lambda row : row.temperature_surface).histogram(5)
(start_vals, counts) = hist_rdd

zipped = zip(start_vals, counts)
type(zipped)
hist_df = spark.createDataFrame(zipped)
hist_df.createOrReplaceTempView("histogramTable")

# rdd = df.rdd.map(lambda row : row.temperature_surface)
# help(spark.createDataFrame)
# spark.createDataFrame(rdd, FloatType())
# spark.createDataFrame(hist_rdd)

DataFrame[_1: double, _2: bigint]

In [20]:
# Let's get all the snow cover values:
snow = spark.sql("SELECT snow_cover_surface FROM TEMP_DF").collect()
# .collect() gives us a list of rows. Let's grab the first 10:
for i in range(10):
    print(snow[i])


# What's the maximum value?
snowmax = spark.sql("SELECT MAX(snow_cover_surface) as maxval FROM TEMP_DF").collect()

print('Max val observed:', snowmax)


Row(snow_cover_surface=0.0)
Row(snow_cover_surface=0.0)
Row(snow_cover_surface=0.0)
Row(snow_cover_surface=100.0)
Row(snow_cover_surface=0.0)
Row(snow_cover_surface=0.0)
Row(snow_cover_surface=100.0)
Row(snow_cover_surface=0.0)
Row(snow_cover_surface=0.0)
Row(snow_cover_surface=0.0)
Max val observed: [Row(maxval=100.0)]


#### So Snowy
One location where it is snowing all year, i.e. where it is snowing for every record in its timeseries is {location}.


In [8]:
# gdf = df.groupBy('Geohash')
# gdf.agg(sum('categorical_snow_yes1_no0_surface'), count('*')).orderBy('sum(categorical_snow_yes1_no0_surface)', ascending = False).collect()

so_snowy = spark.sql("SELECT geohash, sum, cnt, (sum / cnt) AS div \
            FROM ( \
                SELECT geohash, SUM(categorical_snow_yes1_no0_surface) AS sum, COUNT(*) as cnt \
                FROM nam_tiny \
                GROUP BY(geohash) \
            ) ORDER BY div DESC")
so_snowy.head(10)

[Row(geohash='c1nuq5290jup', sum=1.0, cnt=1, div=1.0),
 Row(geohash='f2w29r4werxb', sum=1.0, cnt=1, div=1.0),
 Row(geohash='f2d5v1jeyp7z', sum=1.0, cnt=1, div=1.0),
 Row(geohash='c6s64488ws80', sum=1.0, cnt=1, div=1.0),
 Row(geohash='f2fh6jpdgv5b', sum=1.0, cnt=1, div=1.0),
 Row(geohash='fccz22w4fytb', sum=1.0, cnt=1, div=1.0),
 Row(geohash='drmg1tprm22p', sum=0.0, cnt=1, div=0.0),
 Row(geohash='8gzfnvh85g0p', sum=0.0, cnt=1, div=0.0),
 Row(geohash='8up5c0e570pz', sum=0.0, cnt=1, div=0.0),
 Row(geohash='dse39p1jkszz', sum=0.0, cnt=1, div=0.0)]

# Analysis

### Strangely Snowy 
Find a location that contains snow while its surroundings do not. Why does this occur? Is it a high mountain peak in a desert?

### Lightning rod
Where are you most likely to be struck by lightning? Use a precision of at least 4 Geohash characters and provide the top 3 locations.

In [38]:
lightning_rod = spark.sql("\
                           SELECT LEFT(geohash, 4) AS geo4, SUM(lightning_surface) / COUNT(*) AS prob_lightning \
                           FROM nam_tiny \
                           GROUP BY geo4 \
                           ORDER BY prob_lightning DESC")

In [42]:
lightning_rod.head(3)

[Row(geo4='dmbz', prob_lightning=1.0),
 Row(geo4='c4xg', prob_lightning=0.0),
 Row(geo4='d5x3', prob_lightning=0.0)]

### Drying out
Choose a region in North America (defined by one or more Geohashes) and determine when its driest month is. This should include a histogram with data from each month.

In [95]:
# from_unixtime(timestamp DIV 1000) as timestring, \

drying_out = spark.sql("\
    SELECT \
        trunc(from_unixtime(timestamp DIV 1000), 'MM') as truncated \
    FROM nam_tiny \
    GROUP BY truncated")

In [96]:
drying_out.show()

+----------+
| truncated|
+----------+
|2015-03-01|
+----------+



### Travel Startup
After graduating from USF, you found a startup that aims to provide personalized travel itineraries using big data analysis. Given your own personal preferences, build a plan for a year of travel across 5 locations. Or, in other words: pick 5 regions. What is the best time of year to visit them based on the dataset?

- One avenue here could be determining the comfort index for a region. You could incorporate several features: not too hot, not too cold, dry, humid, windy, etc. There are several different ways of calculating this available online, and you could also analyze how well your own metrics do.

### Escaping the fog
After becoming rich from your startup, you are looking for the perfect location to build your Bay Area mansion with unobstructed views. Find the locations that are the least foggy and show them on a map.


In [ ]:
spark.sql("\
    SELECT LEFT(geohash) as geo4, SUM(visibility_surface) /  \
    FROM nam_tiny \
    GROUP BY truncated")

### SolarWind, Inc.
You get bored enjoying the amazing views from your mansion, so you start a new company; here, you want to help power companies plan out the locations of solar and wind farms across North America. Locate the top 3 places for solar and wind farms, as well as a combination of both (solar + wind farm). You will report a total of 9 Geohashes as well as their relevant attributes (for example, cloud cover and wind speeds).

# Sampling

We can even create a sample dataset with Spark! Let's create a 10% sample (without replacement)

In [ ]:
samp = df.sample(False, .1)

# Write it out to a file
samp.write.format('csv').save('hdfs://orion12:50000/sampled_output.csv')

In [27]:
bayarea_tiny = spark.sql("\
                           SELECT * \
                           FROM nam_tiny \
                           WHERE LEFT(geohash, 3) IN ('9qb', '9qc', '9q9', '9q8')")


nine_tiny = spark.sql("\
                           SELECT * \
                           FROM nam_tiny \
                           WHERE LEFT(geohash, 1) IN ('9', '8')")

In [29]:
bayarea_small = spark.sql("\
                           SELECT * \
                           FROM nam_small \
                           WHERE LEFT(geohash, 3) IN ('9qb', '9qc', '9q9', '9q8')")

In [8]:
bayarea_full = spark.sql("\
SELECT * \
FROM nam_full \
WHERE LEFT(geohash, 4) IN ( \
 '9q8x', '9q8z', '9q8y', '9q8v', '9q8u', '9q8g', \
 '9q9p', '9q9n', '9q9j', '9q9h', '9q95', '9q9r', '9q9q', '9q9m', '9q9k', '9q97', '9q9e', \
 '9qb3', '9q95', '9qb6', '9qb7', '9qb8', '9qb9', '9qbb', '9qbc', '9qbd', '9qbe', '9qbf', '9qbg', '9qbh', '9qbk', '9qbs', \
 '9qc0', '9qc1', '9qc2', '9qc3', '9qc4', '9qc5', '9qc6', '9qch' \
)")

In [ ]:
bayarea_full.write.format('csv').save('hdfs://orion11:15000/nam_bayarea')

In [ ]:
bayarea_full.createOrReplaceTempView("nam_bayarea")